# How to call text-to-text DIAL applications

From [this notebook](https://github.com/epam/ai-dial/blob/main/dial-cookbook/examples/how_to_call_text_to_text_applications.ipynb), you can learn how to call text-to-text DIAL applications via [DIAL API chat/completions](https://epam-rail.com/dial_api#/paths/~1openai~1deployments~1%7BDeployment%20Name%7D~1chat~1completions/post) call.

For this example, we use a sample text-to-text application called **Echo**, which returns the content of the last user message.

## Setup

**Step 1**: install the necessary dependencies and import the libraries we are going to use.

In [ ]:
!pip install requests==2.31.0
!pip install openai==1.9.0
!pip install langchain-openai==0.0.3

In [4]:
import requests
import openai
import langchain_openai

**Step 2**: run `docker compose` in a separate terminal to start the **DIAL Core** server locally along with the **Echo** application.

```sh
(cd ..; docker compose up core echo)
```

**Step 3**: configure DIAL Core URL: it will be `http://localhost:8080`, if the DIAL Core is run locally.

In [3]:
import os
dial_url = os.environ.get("DIAL_URL", "http://localhost:8080")
os.environ["DIAL_URL"] = dial_url

## Using Curl

* The Echo application deployment is called `echo`.
* The local DIAL Core server URL is `dial_url`.
* The OpenAI API version we are going to use is `2023-12-01-preview`.

Therefore, the Echo application is accessible via the following URL:

```
http://${DIAL_URL}/openai/deployments/echo/chat/completions?api-version=2023-12-01-preview
```

The corresponding curl command with a singe message in the request is:

In [4]:
!curl -X POST "${DIAL_URL}/openai/deployments/echo/chat/completions?api-version=2023-12-01-preview" \
  -H "Api-Key:dial_api_key" \
  -H "Content-Type:application/json" \
  -d '{"messages": [{"role": "user", "content": "Hello world!"}]}'

{"choices":[{"index":0,"finish_reason":"stop","message":{"role":"assistant","content":"Hello world!"}}],"usage":null,"id":"544c5811-0363-4e45-996a-e03e70972c53","created":1706540436,"object":"chat.completion"}

## Using Python library Requests


Let's make an HTTP request using the Python library `requests` and make sure the output message matches the message in the request.

The arguments are identical to the curl command above.

Let's call the application in the **non-streaming** mode first:


In [5]:
response = requests.post(
    f"{dial_url}/openai/deployments/echo/chat/completions?api-version=2023-12-01-preview",
    headers={"Api-Key": "dial_api_key"},
    json={"messages": [{"role": "user", "content": "Hello world!"}]},
)
body = response.json()
display(body)
completion = body["choices"][0]["message"]["content"]
print(f"Completion: {completion!r}")
assert completion == "Hello world!", "Unexpected completion"

{'choices': [{'index': 0,
   'finish_reason': 'stop',
   'message': {'role': 'assistant', 'content': 'Hello world!'}}],
 'usage': None,
 'id': '36bece7b-c81b-4415-ad1a-011504bbeed5',
 'created': 1706540436,
 'object': 'chat.completion'}

Completion: 'Hello world!'


When **streaming is enabled**, the chat completion returns a sequence of messages, each containing a chunk of a generated response:

In [6]:
response = requests.post(
    f"{dial_url}/openai/deployments/echo/chat/completions?api-version=2023-12-01-preview",
    headers={"Api-Key": "dial_api_key"},
    json={"messages": [{"role": "user", "content": "Hello world!"}], "stream": True},
)
for chunk in response.iter_lines():
    print(chunk)

b'data: {"choices":[{"index":0,"finish_reason":null,"delta":{"role":"assistant"}}],"usage":null,"id":"2a75a848-1ba7-427d-b9a7-75cd4b3f6757","created":1706540436,"object":"chat.completion.chunk"}'
b''
b'data: {"choices":[{"index":0,"finish_reason":null,"delta":{"content":"Hello world!"}}],"usage":null,"id":"2a75a848-1ba7-427d-b9a7-75cd4b3f6757","created":1706540436,"object":"chat.completion.chunk"}'
b''
b'data: {"choices":[{"index":0,"finish_reason":"stop","delta":{}}],"usage":null,"id":"2a75a848-1ba7-427d-b9a7-75cd4b3f6757","created":1706540436,"object":"chat.completion.chunk"}'
b''
b'data: [DONE]'
b''


## Using OpenAI Python SDK

The DIAL deployment could be called using [OpenAI Python SDK](https://pypi.org/project/openai/) as well.


In [7]:
openai_client = openai.AzureOpenAI(
    azure_endpoint=dial_url,
    azure_deployment="echo",
    api_key="dial_api_key",
    api_version="2023-12-01-preview",
)

Let’s call the application in the **non-streaming** mode:

In [8]:

chat_completion = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hello world!",
        }
    ],
    model="echo",
)
print(chat_completion)
completion = chat_completion.choices[0].message.content
print(f"Completion: {completion!r}")
assert completion == "Hello world!", "Unexpected completion"

ChatCompletion(id='17837de1-bdc6-4677-9117-acc4a38bfda9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello world!', role='assistant', function_call=None, tool_calls=None))], created=1706540436, model=None, object='chat.completion', system_fingerprint=None, usage=None)
Completion: 'Hello world!'


Let’s call the application in the **streaming** mode:

In [9]:
chat_completion = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hello world!",
        }
    ],
    stream=True,
    model="echo",
)
completion = ""
for chunk in chat_completion:
    print(chunk)
    content = chunk.choices[0].delta.content
    if content:
        completion += content
print(f"Completion: {completion!r}")
assert completion == "Hello world!", "Unexpected completion"

ChatCompletionChunk(id='87cfdb6f-64a0-4a4e-9380-f335c915784c', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1706540436, model=None, object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='87cfdb6f-64a0-4a4e-9380-f335c915784c', choices=[Choice(delta=ChoiceDelta(content='Hello world!', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1706540436, model=None, object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='87cfdb6f-64a0-4a4e-9380-f335c915784c', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role=None, tool_calls=None), finish_reason='stop', index=0, logprobs=None)], created=1706540436, model=None, object='chat.completion.chunk', system_fingerprint=None, usage=None)
Completion: 'Hello world!'


## Using LangChain

Let's call the application via the [LangChain](https://pypi.org/project/langchain-openai/) library.

In [10]:
from langchain_core.messages import HumanMessage

llm = langchain_openai.AzureChatOpenAI(
    azure_endpoint=dial_url,
    azure_deployment="echo",
    api_key="dial_api_key",
    api_version="2023-12-01-preview",
)

Let’s call the application in the **non-streaming** mode:

In [11]:
output = llm.generate(messages=[[HumanMessage(content="Hello world!")]])
print(output)
completion = output.generations[0][0].text
print(f"Completion: {completion!r}")
assert completion == "Hello world!", "Unexpected completion"

generations=[[ChatGeneration(text='Hello world!', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Hello world!'))]] llm_output={'token_usage': {}, 'model_name': 'gpt-3.5-turbo'} run=[RunInfo(run_id=UUID('424d3cb8-f89b-4786-9403-ae32f0828eb6'))]
Completion: 'Hello world!'


Let’s call the application in the **streaming** mode:

In [12]:
output = llm.stream(input=[HumanMessage(content="Hello world!")])
completion = ""
for chunk in output:
    print(chunk.dict())
    completion += chunk.content
print(f"Completion: {completion!r}")
assert completion == "Hello world!", "Unexpected completion"

{'content': '', 'additional_kwargs': {}, 'type': 'AIMessageChunk', 'example': False}
{'content': 'Hello world!', 'additional_kwargs': {}, 'type': 'AIMessageChunk', 'example': False}
{'content': '', 'additional_kwargs': {}, 'type': 'AIMessageChunk', 'example': False}
Completion: 'Hello world!'
